# MA5701 Optimización no lineal: Tarea 3
**Fecha de entrega:** 4 de Julio a las 23:59 hrs.

**Profesor:** Jorge G. Amaya A. 

**Auxiliar:** Aldo Gutiérrez Concha. 

**Ayudantes:** Carolina Chiu y Mariano Vazquez.

**Autor:** Felipe Urrutia Vargas

In [42]:
from scipy.optimize import linprog
import numpy as np
!pip install numdifftools
import numdifftools as nd
import re
import pandas as pd
import time

#  Método de direcciones admisibles (Zoutendijk)

Resolver el problema de optimización no lineal:
$$(P) \quad \begin{cases} \min &f (x) \\
s.a. &Ax \leq b \\
&Ex = e
\end{cases}$$

## Paso 0

In [2]:
def step0(xk, A, b, E=None, e=None):
    # Step 0: Check factibilidad
    feasible = np.all(A@xk <= b)
    if str(E) != "None":
        feasible = feasible and np.all(np.isclose(E@xk, e))
    return feasible

## Paso 1

In [3]:
def step1(xk, A, b):
    # Step 1
    index_A1 = np.isclose(A@xk, b)
    index_A2 = ~index_A1

    A1 = A[index_A1]
    A2 = A[index_A2]

    b1 = b[index_A1]
    b2 = b[index_A2]
    
    return A1, A2, b1, b2

## Paso 2

In [4]:
def step2(obj, A1, E, eps):
    # Step 2
    lhs_ineq = A1
    rhs_ineq = np.zeros(A1.shape[0]) if str(A1) != "None" else None

    lhs_eq = E
    rhs_eq = np.zeros(E.shape[0]) if str(E) != "None" else None

    bnd = [(-1, 1) for _ in range(obj.shape[0])] 

    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
                  A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
                  method="simplex")
    dk = opt.x
    stop = abs(np.dot(obj, dk))<eps
    return dk, stop

## Paso 3

In [9]:
def step3(f, xk, dk, A2, b2, eps, N=10*6, sgm=0.5):
    # Step 3
    # Lambda maximo
    dom = A2@dk>0
    bar_lmbdk = np.min((b2-A2@xk)[dom]/(A2@dk)[dom]) if len(dom) != 0 else float("inf")

    # Paso de armijo
    t = np.linspace(0, np.min([bar_lmbdk, 10*6]), N)
    h = 1/(N-1)

    d_phi_0 = nd.Gradient(lambda t: f( xk + dk*t ))([0])

    dom = np.where(f( (xk + dk * t[:, None]).T ) <= f(xk) + sgm*d_phi_0*t)[0]
    ix_max = np.max(dom)
    lmbdk = ix_max * h * bar_lmbdk

    # Nuevo paso
    xk1 = xk + lmbdk * dk
    stop = lmbdk<eps
    return xk1, stop

## Clase Zoutendijk

In [84]:
class Zoutendijk(object):
    
    def __init__(self, fun: str, const: list):
        self.fun = fun
        self.const = const
        
        self.set_vars()
        self.set_f()
        self.set_A_b_E_e()
        print(f"""-------------- Model Size --------------
   Variables    :  {self.n}
   Constraints  :  {self.m}
   Eq. consts.  :  {self.m_E}
   Ineq. consts.:  {self.m_A}
 ------------------------------------------""")
    
    def set_vars(self):
        self.vars = set(re.findall("x\d", self.fun))
        for c in self.const:
            self.vars = self.vars.union(set(re.findall("x\d", c)))   
        self.vars = sorted(list(self.vars))
        self.dic_vars = {var: f"x[{k}]" for k, var in enumerate(self.vars)}
    
    def set_f(self):
        self.rename_fun = self.fun
        for k, v in self.dic_vars.items():
            self.rename_fun = self.rename_fun.replace(k, v)
    
    def set_A_b_E_e(self):
        self.n = len(self.dic_vars)
        self.m = len(self.const)
        self.m_A = sum(int("<" in c or ">" in c) for c in self.const)
        self.m_E = self.m-self.m_A

        self.A = None
        self.E = None

        self.b = None
        self.e = None

        if self.m_A:
            self.A = np.zeros((self.m_A, self.n))
            self.b = np.zeros(self.m_A)
        if self.m_E:
            self.E = np.zeros((self.m_E, self.n))
            self.e = np.zeros(self.m_E)

        i_A = 0
        i_E = 0
        for c in self.const:
            if "<" in c or ">" in c:
                sgn = 1 if "<" in c else -1
                left, right = c.split("<=") if sgn>0 else c.split(">=")
                self.b[i_A] = sgn*float(right.strip())

                t_2 = ""
                t_1 = ""
                t = ""
                for t1 in left.strip().split():
                    if t1 in self.dic_vars.keys():
                        v = self.dic_vars[t1]
                        j = int(re.findall("\d", v)[0])

                        if t_2 == t_1 == t == "":
                            self.A[i_A, j] = sgn
                        elif t_2 == t_1 == "" and t == "-":
                            self.A[i_A, j] = sgn*(-1)
                        elif t_2 in ["", "+"] and t == "*":
                            self.A[i_A, j] = sgn*float(t_1)
                        elif t_2 == "-" and t == "*":
                            self.A[i_A, j] = sgn*float(t_1)*(-1)
                        elif t_1 in self.dic_vars.keys() and t == "+":
                            self.A[i_A, j] = sgn
                        elif t_1 in self.dic_vars.keys() and t == "-":
                            self.A[i_A, j] = sgn*(-1)
                    t_2 = t_1
                    t_1 = t
                    t = t1
                i_A+=1
            else:
                left, right = c.split("=")
                self.e[i_E] = float(right.strip())

                t_2 = ""
                t_1 = ""
                t = ""
                for t1 in left.strip().split():
                    if t1 in self.dic_vars.keys():
                        v = self.dic_vars[t1]
                        j = int(re.findall("\d", v)[0])

                        if t_2 == t_1 == t == "":
                            self.E[i_E, j] = 1
                        elif t_2 == t_1 == "" and t == "-":
                            self.E[i_E, j] = -1
                        elif t_2 in ["", "+"] and t == "*":
                            self.E[i_E, j] = float(t_1)
                        elif t_2 == "-" and t == "*":
                            self.E[i_E, j] = float(t_1)*(-1)
                        elif t_1 in self.dic_vars.keys() and t == "+":
                            self.E[i_E, j] = 1
                        elif t_1 in self.dic_vars.keys() and t == "-":
                            self.E[i_E, j] = -1
                    t_2 = t_1
                    t_1 = t
                    t = t1
                i_E+=1
                
    def f(self, x):
        return eval(self.rename_fun)
    
    def print_status(self, feasible, xk, f_xk, step=0, k=0):
        print(f"(step={step}, k={k}) Factible: {feasible}")
        print(f"(step={step}, k={k}) xk: {xk}")
        print(f"(step={step}, k={k}) f(xk): {f_xk}\n")
                
    def solve(self, x0, eps=1e-6, N=10*6, sgm=0.5, verbose=True):
        start=time.time()
        # Step 0: Inicializar
        k = 0
        xk = x0
        grad = nd.Gradient(self.f)
        # Step 0: Check factibilidad
        feasible = step0(xk, self.A, self.b, self.E, self.e)
        stop = not feasible
        # Step 0: Imprimir estado
        f_xk = self.f(xk)
        if verbose:
            self.print_status(feasible, xk, f_xk, step=0, k=k)
        self.list_xk = [xk]
        self.list_f_xk = [f_xk]   
        self.list_feasible = [feasible]
        while not stop:
            # Step 1: Descomposicion [A1, A2]
            A1, A2, b1, b2 = step1(xk, self.A, self.b)
            # Step 2: Resolver problema lineal (Dk)
            obj = grad(xk)
            dk, stop_grad = step2(obj, A1, self.E, eps)
            stop = stop_grad
            if not stop:
                # Step 3: Resolver problema (L)
                xk1, stop_lambda = step3(self.f, xk, dk, A2, b2, eps, N=N, sgm=sgm)
                xk = xk1
                feasible = step0(xk, self.A, self.b, self.E, self.e)
                stop = (not feasible) or stop_lambda
                # Step 3: Imprimir estado
                f_xk = self.f(xk)
                if verbose:
                    self.print_status(feasible, xk, f_xk, step=3, k=k)
                k+=1
                self.list_xk.append(xk)
                self.list_f_xk.append(f_xk)
                self.list_feasible.append(feasible)
        if k>0:
            print(f""" ---------------------------------------------------
 Solver         :  Zoutendijk (Simplex/Armijo)
 Solution time  :  {time.time()-start:.2f} ms
 Objective      :  {self.list_f_xk[-1]: .4f}
 Stop criterion :  {"f(xk) * dk < eps" if stop_grad else "lmbdk < eps" if stop_lambda else "Unfeasible" if not feasible else "?"}
 {"Successful solution" if feasible else "?"}
 ---------------------------------------------------""")
        if k==0:
            print(f""" ---------------------------------------------------
 Solver         :  Zoutendijk (Simplex/Armijo)
 Objective      :  {self.list_f_xk[-1]: .4f}
 Start point    :  {"Unfeasible" if not feasible else "?"}
 ---------------------------------------------------""")
                
    def report(self, feasible=False):
        df = pd.DataFrame(self.list_xk, columns=self.dic_vars.keys(), index=pd.Series(range(len(self.list_xk)), name="Iter. k"))
        df["f(xk)"] = self.list_f_xk
        if feasible:
            df["Factible"] = self.list_feasible
        return df

## Problema 1

$$(P_1) \quad \begin{cases} \min &8(x_1-6)^2+(x_2-2)^4 \\
s.a. &-x_1+2x_2 \leq 4 \\
&3x_1+2x_2 \leq 12 \\
&x_i \geq 0, \quad i=1,2 \\
\end{cases}$$

In [85]:
%%time
# Inicializar datos del problema
problem_1 = Zoutendijk(
    fun="8 * ( x1 - 6 ) ** 2 + ( x2 - 2 ) ** 4", 
    const=[
        "-x1 + 2 * x2 <= 4",
        "3 * x1 + 2 * x2 <= 12",
        "x1 >= 0",
        "x2 >= 0"
    ]
)
# Resolver problema
problem_1.solve(
    x0=[0, 2], 
    eps=1e-6, 
    N=10*9, 
    sgm=0.1, 
    verbose=False
)
# Reportar resultados
problem_1.report(feasible=True)[:-1]

-------------- Model Size --------------
   Variables    :  2
   Constraints  :  4
   Eq. consts.  :  0
   Ineq. consts.:  4
 ------------------------------------------
 ---------------------------------------------------
 Solver         :  Zoutendijk (Simplex/Armijo)
 Solution time  :  0.06 ms
 Objective      :   46.9050
 Stop criterion :  lmbdk < eps
 Successful solution
 ---------------------------------------------------
Wall time: 60.1 ms


,x1,x2,f(xk),Factible
Iter. k,,,,
0,0.000000,2.000000,288.000000,True
1,2.666667,2.000000,88.888889,True
2,4.000000,0.000000,48.000000,True
3,3.745318,0.382022,47.521860,True
4,3.942768,0.085848,47.282363,True
5,3.799386,0.300921,47.075631,True
6,3.903074,0.145389,47.007514,True
7,3.833613,0.249580,46.933764,True
8,3.876612,0.185082,46.920170,True


https://www.wolframalpha.com/input?i=minimize+8%28x+%E2%88%92+6%29%5E2+%2B+%28y+%E2%88%92+2%29%5E4+subject+to+%E2%88%92x+%2B+2y+%3C%3D+4%2C++3x+%2B+2y+%3C%3D+12%2C+x+%3E%3D+0%2C+y+%3E%3D0

## Problema 2

$$(P_2) \quad \begin{cases} \min &x_1^4-2x_2^2+10x_1x_2^2+x_4^2 \\
s.a. &x_1+x_2-x_3 = 1 \\
&x_1+x_4 = 4 \\
&x_1-x_2=0 \\
&x_i \geq 0, \quad i=1,2 \\
\end{cases}$$

In [87]:
%%time
# Inicializar datos del problema
problem_2 = Zoutendijk(
    fun="x1 ** 4 - 2 * x2 ** 2 + 10 * x1 * ( x2 ** 2 ) + x4 ** 2", 
    const=[
        "x1 + x2 - x3 = 1",
        "x1 + x4 = 4",
        "x1 - x2 = 0",
        "x1 >= 0",
        "x2 >= 0",
        "x3 >= 0",
        "x4 >= 0"
    ]
)
# Resolver problema
problem_2.solve(
    x0=[2, 2, 3, 2], 
    eps=1e-6, 
    N=10*6, 
    sgm=0.5, 
    verbose=False
)
# Reportar resultados
problem_2.report(feasible=True)[:-1]

-------------- Model Size --------------
   Variables    :  4
   Constraints  :  7
   Eq. consts.  :  3
   Ineq. consts.:  4
 ------------------------------------------
 ---------------------------------------------------
 Solver         :  Zoutendijk (Simplex/Armijo)
 Solution time  :  0.06 ms
 Objective      :   13.0468
 Stop criterion :  lmbdk < eps
 Successful solution
 ---------------------------------------------------
Wall time: 57.1 ms


,x1,x2,x3,x4,f(xk),Factible
Iter. k,,,,,,
0,2.000000,2.000000,3.000000,2.000000,92.000000,True
1,0.932203,0.932203,0.864407,3.067797,16.529413,True
2,0.580580,0.580580,0.161161,3.419420,13.088885,True
3,0.532778,0.532778,0.065557,3.467222,13.046799,True
4,0.531667,0.531667,0.063335,3.468333,13.046758,True
5,0.531131,0.531131,0.062261,3.468869,13.046754,True


https://www.wolframalpha.com/input?i=minimize+x%5E4+-+2*y%5E2%2B10*x*y%5E2%2Bw%5E2%2B0.0000001*z+subject+to+x+%2B+y-+z%3D+1%2C+x%2B+w%3D+4%2C+x-+y%3D+0%2C+x%3E%3D0%2C+y%3E%3D0%2C+z%3E%3D0%2C+w%3E%3D0